In [1]:
!pip install yfinance

     |████████████████████████████████| 5.5MB 11.9MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.59-py2.py3-none-any.whl size=23442 sha256=fd564bb555027156b88ec2ac8a0b5972b2460b3becfe3195af1efaf4c224427c
  Stored in directory: /root/.cache/pip/wheels/f8/2a/0f/4b5a86e1d52e451757eb6bc17fd899629f0925c777741b6d04
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [2]:
import warnings
warnings.filterwarnings('ignore')

import yfinance as yf
import pandas as pd
import numpy as np
import altair as alt
import datetime
import project_functions2 as pf

from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
stock_list = ['AMZN', 'AAPL', 'FB','GOOG', 'MSFT', 'TSLA']

In [5]:
stock_objects = {}
for stock in stock_list:
    stock_objects[stock] = yf.Ticker(stock)

In [6]:
def trading_sim_LRM(stock_objects, split_time, time_shift):
    columns = ['Date', 'Fund Value', 'Cash']
    stock_names = []
    proj_buy = 0
    curr_cash = 10000
    curr_shares = {}
    drop_list = [ 'Volume', 'Dividends', 'Stock Splits',
                 '5 Day Open Mean', '5 Day High Mean', '5 Day Low Mean',
                 '5 Day Close Mean', '5 Day Volume Mean', '5 Day Open Var',
                 '5 Day High Var', '5 Day Low Var', '5 Day Close Var',
                 '5 Day Volume Var','10 Day Open Mean', '10 Day High Mean', 
                 '10 Day Low Mean','10 Day Close Mean', '10 Day Volume Mean', 
                 '10 Day High Var', '10 Day Low Var', '10 Day Close Var',
                 '10 Day Volume Var', '10 Day High', '10 Day Low', 
                 '20 Day Open Mean', '20 Day High Mean', '20 Day Low Mean',
                 '20 Day Close Mean', '20 Day Volume Mean', '20 Day Open Var',
                 '20 Day High Var', '20 Day Low Var', '20 Day Close Var',
                 '20 Day Volume Var', '10 Day Open Var']
    
    stock_dfs = {}
    for key in stock_objects:
        stock_dfs[key] = stock_objects[key].history(start='1990-01-01', end='2021-03-31')
        
    stock_investing = {}
    for key in stock_objects:
        stock_investing[key] = pd.read_csv('/content/drive/MyDrive/SENG474_Project/data/sentiment/investing_'+key+'_sentiment.csv')
        stock_investing[key].set_index('date', inplace=True)
    stock_stocks = {}
    for key in stock_objects:
        stock_stocks[key] = pd.read_csv('/content/drive/MyDrive/SENG474_Project/data/sentiment/stocks_'+key+'_sentiment.csv')
        stock_stocks[key].set_index('date', inplace=True)
    
    for key in stock_dfs:
        stock_dfs[key] = pf.rolling_aves(stock_dfs[key])
        stock_dfs[key].drop(drop_list, axis=1, inplace=True)
        stock_dfs[key] = stock_dfs[key].merge(stock_investing[key], how='left', left_index=True, right_index=True)
        stock_dfs[key] = stock_dfs[key].merge(stock_stocks[key], how='left', left_index=True, right_index=True)
        stock_dfs[key].replace([np.inf, -np.inf], np.nan, inplace=True)
        stock_dfs[key].fillna(0, inplace=True)
        stock_dfs[key] = pf.future_low_setup(stock_dfs[key], 1)
        stock_dfs[key] = pf.future_close_setup(stock_dfs[key], time_shift)

    combine_df = pf.combiner(stock_dfs)
        
    test_dfs = stock_dfs
    
    for key in test_dfs:
        test_dfs[key] = test_dfs[key].tail(split_time)
        curr_shares[key] = 0
        stock_names.append(key)
    
    columns = columns + stock_names
    cash_df = pd.DataFrame(columns=columns)
    curr_line = [combine_df.index[int(len(combine_df) - (split_time * len(stock_dfs)))], curr_cash, curr_cash] + len(stock_names)*[0]
    cash_df.loc[len(cash_df)] = curr_line
    
    while split_time > time_shift:
        #print('Days left: ' +str(split_time))
        max_stock = ''
        max_stock_gain = 0
        split_mark = int(len(combine_df) - (split_time * len(stock_dfs)))
        
        X_train, y_train, X_test, y_test = pf.multi_stock_train_test_split(combine_df, split_time, stock_dfs)
        low_train = X_train.iloc[:,-1]
        low_test = X_test.iloc[:,-1]
        X_train.drop(['Low in 1 Days'], axis=1, inplace=True)
        X_test.drop(['Low in 1 Days'], axis=1, inplace=True)
        stock_model = LinearRegression().fit(X_train, y_train)
        low_model = LinearRegression().fit(X_train, low_train)
        
        for key in test_dfs:
            X = test_dfs[key].iloc[:,:-2]
            stock_pred = stock_model.predict(X.head(1))
            stock_pred = (float(stock_pred) - X['Close'][0]) / X['Close'][0]
            low_pred = low_model.predict(X.head(1))
            if stock_pred > max_stock_gain and low_pred > 0:
                max_stock = key
                max_stock_gain = stock_pred
                max_stock_low_pred = low_pred
        
        for key in test_dfs:
            test_dfs[key] = test_dfs[key].iloc[1:]
        
        
        if max_stock_gain > 0:
            if test_dfs[max_stock]['Open'][0] < max_stock_low_pred:
                proj_buy = test_dfs[max_stock]['Open'][0]
            else:    
                proj_buy = float((test_dfs[max_stock]['Open'][0] - max_stock_low_pred)*0.5 + test_dfs[max_stock]['Low'][0])
            #print(test_dfs[max_stock].index[0])
            #if max_stock_low_pred >= test_dfs[max_stock]['Open'][0]:
            #   print('Buy at open')
            #print(float(proj_buy - test_dfs[max_stock]['Low'][0]))
            if curr_shares[max_stock] == 0:
                for key in curr_shares:
                    if curr_shares[key] > 0:
                        curr_cash -= 6.95
                    curr_cash += curr_shares[key]*test_dfs[key]['Open'][0]
                    curr_shares[key] = 0

                if proj_buy >= test_dfs[max_stock]['Low'][0]:
                    curr_cash -= 6.95
                    curr_shares[max_stock] = curr_cash // proj_buy
                    curr_cash -= curr_shares[max_stock]*proj_buy
            elif proj_buy >= test_dfs[max_stock]['Low'][0]:
                if curr_cash > 70:
                    curr_cash -= 6.95
                    curr_shares[max_stock] += curr_cash // proj_buy
                    curr_cash -= (curr_cash // proj_buy) * proj_buy

            '''if proj_buy*0.9 > test_dfs[max_stock]['Low'][0]:
                if curr_shares[max_stock] > 0:
                    curr_cash -= 6.95
                curr_cash += curr_shares[max_stock]*proj_buy*0.9
                curr_shares[max_stock] = 0'''

        else:
            for key in curr_shares:
                if curr_shares[key] > 0:
                    curr_cash -= 6.95
                curr_cash += curr_shares[key]*test_dfs[key]['Open'][0]
                curr_shares[key] = 0
        
        curr_line = [X_test.index[len(stock_names)], curr_cash ,curr_cash] + len(stock_names)*[0]
        cash_df.loc[len(cash_df)] = curr_line
        for key in curr_shares:
            cash_df.iloc[-1, cash_df.columns.get_loc(key)] = curr_shares[key]
            cash_df.iloc[-1, cash_df.columns.get_loc('Fund Value')] += curr_shares[key]*test_dfs[key]['Open'][0]
        for key in test_dfs:
            test_dfs[key] = test_dfs[key].iloc[time_shift-1:]
        
        split_time -= time_shift

    return cash_df

In [7]:
five_day_fund_df = trading_sim_LRM(stock_objects, 292, 5)
five_day_fund_df.set_index('Date', inplace=True)
five_day_fund_df

,Fund Value,Cash,AMZN,AAPL,FB,GOOG,MSFT,TSLA
Date,,,,,,,,
2020-02-03,10000,10000,0,0,0,0,0,0
2020-02-04,9993.05,1250.63,0,0,0,6,0,0
2020-02-11,10307.6,8.28055,0,0,49,0,0,0
2020-02-19,10443.1,99.8898,0,0,0,0,0,56
2020-02-26,8755.67,19.5307,0,0,0,0,52,0
2020-03-04,8796.82,87.6714,0,0,0,0,0,57
2020-03-11,7556.39,90.019,0,0,0,0,48,0
2020-03-18,6633.48,20.4824,0,0,0,0,0,85
2020-03-25,9289.73,20.4824,0,0,0,0,0,85


In [11]:
index = yf.Ticker('^IXIC')
index_df = index.history(start='2019-01-01')
index_df

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2019-01-02,6506.910156,6693.709961,6506.879883,6665.939941,2261800000,0,0
2019-01-03,6584.770020,6600.209961,6457.129883,6463.500000,2607290000,0,0
2019-01-04,6567.140137,6760.689941,6554.240234,6738.859863,2579550000,0,0
2019-01-07,6757.529785,6855.600098,6741.399902,6823.470215,2507550000,0,0
2019-01-08,6893.439941,6909.580078,6795.859863,6897.000000,2380290000,0,0
...,...,...,...,...,...,...,...
2021-04-05,13594.900391,13720.580078,13582.759766,13705.589844,4439970000,0,0
2021-04-06,13681.669922,13776.709961,13674.280273,13698.379883,4038510000,0,0
2021-04-07,13675.299805,13733.030273,13653.589844,13688.839844,4078990000,0,0


In [12]:
results_df = five_day_fund_df.merge(index_df, how='left', left_index=True, right_index=True)
results_df.drop(results_df.columns.difference(['Fund Value','Close']), 1, inplace=True)
results_df.rename(columns={'Close': 'NASDAQ Composite'}, inplace=True)
nas_open = results_df['NASDAQ Composite'][0]
results_df['NASDAQ Composite'] = results_df['NASDAQ Composite']*10000 / nas_open
results_df.reset_index(inplace=True)
print(results_df)
results_df = results_df.melt('Date', var_name='Company', value_name='Price')

# Plots Results
line_plot = alt.Chart(results_df).mark_line().encode(
    x = 'Date',
    y = 'Price',
    color = 'Company'
).properties(
    title='Big 6 Linear Regression Simulation Feb 2020-April 2021'
)
line_plot

         Date Fund Value  NASDAQ Composite
0  2020-02-03      10000      10000.000000
1  2020-02-04    9993.05      10209.814445
2  2020-02-11    10307.6      10394.181232
3  2020-02-19    10443.1      10586.386087
4  2020-02-26    8755.67       9684.452191
5  2020-03-04    8796.82       9724.685082
6  2020-03-11    7556.39       8575.117508
7  2020-03-18    6633.48       7537.515420
8  2020-03-25    9289.73       7962.882539
9  2020-04-01    8574.58       7937.304298
10 2020-04-08    9288.93       8724.846940
11 2020-04-16    12200.2       9200.897181
12 2020-04-23    12610.1       9160.339942
13 2020-04-30    14171.6       9586.073533
14 2020-05-07    14490.7       9683.244310
15 2020-05-14    14049.5       9644.487836
16 2020-05-21    14686.8      10012.378946
17 2020-05-29    14412.8      10233.430799
18 2020-06-05    15660.8      10583.043614
19 2020-06-12    16585.3      10340.122465
20 2020-06-19    16881.7      10725.429398
21 2020-06-26      16569      10521.728078
22 2020-07-

alt.Chart(...)

In [8]:
fund_df = trading_sim_LRM(stock_objects, 292, 1)
fund_df.set_index('Date', inplace=True)
fund_df

,Fund Value,Cash,AMZN,AAPL,FB,GOOG,MSFT,TSLA
Date,,,,,,,,
2020-02-03,10000,10000,0,0,0,0,0,0
2020-02-04,9972.05,54.2881,0,0,48,0,0,0
2020-02-05,10263.7,14.9995,0,128,0,0,0,0
2020-02-06,10233.6,14.9995,0,128,0,0,0,0
2020-02-07,10251.5,14.9995,0,128,0,0,0,0
...,...,...,...,...,...,...,...,...
2021-03-24,25003.2,290.573,0,0,0,0,0,37
2021-03-25,22964.6,283.623,0,0,0,0,0,37
2021-03-26,23988.8,55.0142,0,0,86,0,0,0


In [9]:
index = yf.Ticker('^IXIC')
index_df = index.history(start='2019-01-01')
index_df

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2019-01-02,6506.910156,6693.709961,6506.879883,6665.939941,2261800000,0,0
2019-01-03,6584.770020,6600.209961,6457.129883,6463.500000,2607290000,0,0
2019-01-04,6567.140137,6760.689941,6554.240234,6738.859863,2579550000,0,0
2019-01-07,6757.529785,6855.600098,6741.399902,6823.470215,2507550000,0,0
2019-01-08,6893.439941,6909.580078,6795.859863,6897.000000,2380290000,0,0
...,...,...,...,...,...,...,...
2021-04-05,13594.900391,13720.580078,13582.759766,13705.589844,4439970000,0,0
2021-04-06,13681.669922,13776.709961,13674.280273,13698.379883,4038510000,0,0
2021-04-07,13675.299805,13733.030273,13653.589844,13688.839844,4078990000,0,0


In [10]:
results_df = fund_df.merge(index_df, how='left', left_index=True, right_index=True)
results_df.drop(results_df.columns.difference(['Fund Value','Close']), 1, inplace=True)
results_df.rename(columns={'Close': 'NASDAQ Composite'}, inplace=True)
nas_open = results_df['NASDAQ Composite'][0]
results_df['NASDAQ Composite'] = results_df['NASDAQ Composite']*10000 / nas_open
results_df.reset_index(inplace=True)
print(results_df)
results_df = results_df.melt('Date', var_name='Company', value_name='Price')

# Plots Results
line_plot = alt.Chart(results_df).mark_line().encode(
    x = 'Date',
    y = 'Price',
    color = 'Company'
).properties(
    title='Big 6 Linear Regression Simulation Feb 2020-April 2021'
)
line_plot

          Date Fund Value  NASDAQ Composite
0   2020-02-03      10000      10000.000000
1   2020-02-04    9972.05      10209.814445
2   2020-02-05    10263.7      10253.714158
3   2020-02-06    10233.6      10322.157987
4   2020-02-07    10251.5      10266.471159
..         ...        ...               ...
287 2021-03-24    25003.2      13977.493802
288 2021-03-25    22964.6      13994.521040
289 2021-03-26    23988.8      14168.190648
290 2021-03-29    24631.2      14082.914401
291 2021-03-30    24966.5      14067.536285

[292 rows x 3 columns]


alt.Chart(...)